In [1]:
%tb

import os
import os.path as osp 
import re

import renom as rm
import numpy as np
import tqdm
import matplotlib.pyplot as plt

from renom.cuda import set_cuda_active
from renom_img.api.utility.evaluate import Fast_Segmentation_Evaluator
from renom_img.api.utility.misc.display import draw_segment
from renom_img.api.utility.augmentation import Augmentation
from renom_img.api.utility.augmentation.process import Flip, WhiteNoise, ContrastNorm
from PIL import Image

set_cuda_active(False)

No traceback available to show.


In [2]:
def get_unique_label(lbl_list):
    uniq_label = []
    for i in tqdm.trange(len(lbl_list)):
        lbl_file = lbl_list[i]
        lbl = Image.open(lbl_file)
        lbl = np.array(lbl, dtype=np.int32)
        for l in set(lbl.flatten()):
            if not l in uniq_label:
                uniq_label.append(l)
    return uniq_label

In [3]:
train_image_path_list = []
train_annotation_path_list = []
with open("./CamVid/train.txt") as f:
    txt = f.readlines()
    txt = [line.replace("/SegNet/","./").split(" ") for line in txt]
    for i in range(len(txt)):
        train_image_path_list.append(txt[i][0])
        train_annotation_path_list.append(txt[i][1].strip())

In [4]:
valid_image_path_list = []
valid_annotation_path_list = []
with open("./CamVid/val.txt") as f:
    txt = f.readlines()
    txt = [line.replace("/SegNet/","./").split(" ") for line in txt]
    for i in range(len(txt)):
        valid_image_path_list.append(txt[i][0])
        valid_annotation_path_list.append(txt[i][1].strip())

In [5]:
test_image_path_list = []
test_annotation_path_list = []
with open("./CamVid/test.txt") as f:
    txt = f.readlines()
    txt = [line.replace("/SegNet/","./").split(" ") for line in txt]
    for i in range(len(txt)):
        test_image_path_list.append(txt[i][0])
        test_annotation_path_list.append(txt[i][1].strip())       

In [6]:
uniq_label = get_unique_label(train_annotation_path_list)
uniq_label_valid = get_unique_label(valid_annotation_path_list)

for l in uniq_label_valid:
    if not l in uniq_label:
        uniq_label.append(l)
uniq_label.sort()

class_map = np.arange(len(uniq_label))

100%|██████████| 101/101 [00:01<00:00, 52.13it/s]


In [16]:
from renom_img.api.segmentation.fcn import FCN8s
model_class = FCN8s(class_map=class_map)

TypeError: __init__() got an unexpected keyword argument 'ignore_bias'

In [10]:
# lbl = model_class.get_label(valid_annotation_path_list, 0)
# check_class = 0
# lbl = (lbl==check_class) * 1.0
# visualize_img = draw_segment(valid_image_path_list[0], lbl.astype(np.int))
# plt.imshow(visualize_img)

In [ ]:
# lbl = model_class.get_label(valid_annotation_path_list, 1)
# check_class = 2
# lbl = (lbl==check_class) * 1.0
# visualize_img = draw_segment(valid_image_path_list[1], lbl.astype(np.int))
# plt.imshow(visualize_img)

In [17]:
class_map = ["sky", "building", "pole", "road", "pavement",
                          "tree", "sign_symbol", "fence", "car", "pedestrian", "bicyclist", "unlabeled"]

In [19]:
augmentation = Augmentation([
    Flip(),
    WhiteNoise(),
    ContrastNorm([0.5, 1.0])
])

In [21]:
# need to specify the path to the VGG weight
model_file = "./weight/vgg16_from_caffe.h5"

# Model
# model_class.load(model_file)
# model_class.set_models(inference=False)

In [22]:
def callback_end_epoch(epoch, model, avg_train_loss_list, avg_valid_loss_list):
    if epoch % 10 == 0:
        # need to specify the path to the save directory which you want to save weights
        model.save("./weight/model_{}_{}.h5".format(epoch, str(avg_valid_loss_list[-1])))

In [ ]:
# model_class.fit(train_img_path_list=train_image_path_list,
#                 train_annotation_list=train_annotation_path_list,
#                 valid_img_path_list=valid_image_path_list,
#                 valid_annotation_list=valid_annotation_path_list,
#                 batch_size=8,
#                 callback_end_epoch=callback_end_epoch,
#                 augmentation=augmentation,
#                 epoch=200)

In [23]:
# need  to specify the path to the directory that you want to save the segmented images
if not osp.exists("./data/segmentation"):
    os.makedirs("./data/segmentation")

In [ ]:
# lbl_preds, lbl_trues = [], []
# for i in tqdm.trange(len(valid_image_path_list)):
#     datum = model_class.get_preprocessed_data(valid_image_path_list, i)
#     x_data = np.expand_dims(datum, axis=0)
#     lbl_true = model_class.get_label(valid_annotation_path_list, i)

#     x = rm.Variable(x_data)
#     score = model_class(x)
#     lbl_pred = np.argmax(score.as_ndarray(), axis=1)

#     lbl_preds.append(lbl_pred)
#     lbl_trues.append(lbl_true)
#     visualize_img = draw_segment(valid_image_path_list[i], lbl_pred)
#     visualize_img.save("./data/segmentation/{}.png".format(str(i)))

## Scratch Training Algorithm

<hr>